# Palabras Relacionadas - Dataset

En el presente notebook se documenta el preprocesamiento y creación del dataset del proyecto.
## Preprocesamiento documentos
- **Documentos a procesar:** son pertenecientes a las distintas materias, según lo especificado en el [plan de Ingeniería en Sistemas de Información 2023, UTN - FRM](https://www.lamanuelsavio.org/wp-content/uploads/2024/02/Plan-Sistemas.pdf).
- **Ubicación de los documentos a procesar:** desde el root del proyecto `/data/raw`.
- **Ubicación de los documentos procesados:** desde el root en `/data/plain`
- **Nombre de los documentos:** consta de la siguiente sintaxis `<nivel materia> - <abreviatura materia> - <título del material>`
    - `<nivel materia>`: año (entero) de cursado según plan de estudios. Ej. `5` para quinto año
    - `<abreviatura materia>`: abreviatura del nombre de la materia. Ej. `SO` para Sistemas Operativos
    - `<título del material>`: nombre original significativo del pdf

Los **documentos a procesar están en formato .pdf**, por lo que se utiliza la librería [pypdf](https://pypdf.readthedocs.io/en/stable/).

In [30]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 38.0 MB/s eta 0:00:0000:0100:01


In [33]:
import fitz
import os
from os import listdir
from os.path import isfile, join

# Seteo el path al root del proyecto
dev_folder = 'dev'
if(os.getcwd().split('/')[-1] == dev_folder):
    os.chdir('../')
print(os.getcwd()) # debugging, debe imprimir el path al root del proyecto

/home/mrbeast/TPI-RNP-Palabras-Relacionadas-ISI


Creamos el método `raw_to_plain` que dados:
- `raw_path`: path de los archivos a procesar
- `plain_path`: path de los archivos procesados 
 
Crea un archivo de texto plano (`.txt`) por cada archivo procesado y lo guarda en el `plain_path`.

In [34]:
def raw_to_plain(raw_path, plain_path):
    """Convert PDF files to plain text using PyMuPDF.
    
    Args:
        raw_path (str): Directory containing PDF files
        plain_path (str): Directory where to save plain text files
    """
    raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]
    
    for file in raw_files:
        print("\033[94mConvirtiendo archivo: " + file + "\033[0m")
        
        try:
            # Open PDF with PyMuPDF
            doc = fitz.open(f'{raw_path}/{file}.pdf')
            
            # Process each page
            with open(f'{plain_path}/{file}.txt', 'w', encoding='utf-8') as plain_file:
                for page in doc:
                    # Extract text with better line breaks
                    text = page.get_text("text")  # "text" mode preserves reading order
                    plain_file.write(text)
                    
            doc.close()
            
        except Exception as e:
            print(f"\033[91mError processing {file}: {str(e)}\033[0m")


In [35]:
raw_path = "./data/raw"
plain_path = "./data/plain"

raw_to_plain(raw_path, plain_path)

Convirtiendo archivo: 4 - AS - Implementación de un Data Center
Convirtiendo archivo: 4 - RD - Kurose-Ross
Convirtiendo archivo: 5 - SSI - guia_ciberseguridad_gestion_riesgos_metad
Convirtiendo archivo: 4 - AS - Sniffers_y_escaneo_de_puertos
Convirtiendo archivo: 4 - AS - Backups_raids
Convirtiendo archivo: 4 - IO - Breve Resumen ANALISIS SENSIBILIDAD
Convirtiendo archivo: 1 - SyO - Resumen SyO U5
Convirtiendo archivo: 5 - SSI - Magerit_v3_libro1_metodo
Convirtiendo archivo: 3 - AP - AdProy_2_Trabajo en Equipo_2022
Convirtiendo archivo: 5 - SSI - DE641-2021_Anexo
Convirtiendo archivo: 1 - SyO - 2)Recopilacion de la informacion
Convirtiendo archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Convirtiendo archivo: 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion
Convirtiendo archivo: 5 - SSI - Resumen U2 - Disposición ONTI 1 2015
Convirtiendo archivo: 3 - BD - caselli_manual-de-base-de-datos-2019
Convirtiendo archivo: 2 - SO - Stallings
Convirtiendo archivo:

In [37]:
def get_metrics(plain_path, split_char='-'):
    metrics = {}

    plain_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]

    for file in plain_files:

        sf = file.split(split_char)
        anio = sf[0].strip()
        materia = sf[1].strip()

        if(not anio in metrics):
            metrics[anio] = {}
        if(not materia in metrics[anio]):
            metrics[anio][materia] = 0
        metrics[anio][materia] += os.path.getsize(f'{plain_path}/{file}.txt')
    
    return metrics

def print_metrics(metrics):
    for anio, materias in metrics.items():
        print(f"Año {anio}:")
        for materia, tamano in materias.items():
            print(f"\tMateria: {materia} - {str(round(tamano/1000000,2))}MB")
            # TODO: tal vez mostrar la sumatoria de los tamaños
            
print_metrics(get_metrics(plain_path))

Año 1:
	Materia: AyED - 0.07MB
	Materia: SyO - 1.16MB
	Materia: MD - 0.71MB
	Materia: AC - 0.44MB
Año 4:
	Materia: ICS - 0.24MB
	Materia: AS - 2.36MB
	Materia: TA - 0.06MB
	Materia: UXUI - 0.11MB
	Materia: IO - 0.03MB
	Materia: RD - 5.17MB
	Materia: S - 0.02MB
Año 3:
	Materia: DS - 1.28MB
	Materia: BD - 0.26MB
	Materia: AP - 0.14MB
	Materia: CD - 4.85MB
Año 2:
	Materia: SO - 5.8MB
	Materia: AS - 0.59MB
Año 5:
	Materia: SSI - 1.47MB
	Materia: GG - 0.01MB


## Tokenización
A continuación se tokenizan los documentos procesados planos `.txt`, generando un nuevo archivo por cada documento procesado fuente, donde cada línea representa un token. 

Para ello se utiliza la librería [spacy](https://spacy.io/), la cual tomará un rol importante para obtención de tokens, filtrado y transformación de los mismos.

In [23]:
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.3 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [38]:
import spacy
import re

esp = spacy.load("es_core_news_sm")

In [39]:
def tokenize_files(plain_path, tokens_path, banned_tokens):
    raw_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]

    def is_clean_token(token):
        return not (
            token.is_punct or
            token.is_space or
            token.is_stop or
            len(token.text) == 1 or
            token.text in banned_tokens or 
            bool(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', token.text)))

    for f in raw_files:
        print("\033[94mTokenizando archivo: " + f + "\033[0m")

        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = esp.tokenizer(txt)
            with open(f"{tokens_path}/{f}.txt", "wb") as tf:
                for token in tokens:
                    if (is_clean_token(token)):
                        tf.write((token.text + "\n").encode("utf-8"))

In [40]:
tokens_path = "./data/tokens"
banned_tokens = set(["capítulo", "página", "figura", "cap", "ejemplo", "catedra", "mendoza", "argentina", "muñoz", "facchini", "cesari", "xsd", "infoleg"])
tokenize_files(plain_path, tokens_path, banned_tokens)

Tokenizando archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando archivo: 1 - SyO - Resumen SyO U3
Tokenizando archivo: 4 - ICS - 2.-principiosingenieriasoftware
Tokenizando archivo: 3 - DS - MerFNConceptos
Tokenizando archivo: 2 - SO - ResumenSO
Tokenizando archivo: 1 - AyED - Unidad4 (7930)
Tokenizando archivo: 2 - AS - U8 Metodologías Agiles parte a 
Tokenizando archivo: 3 - DS - Libro UML y Patrones - Larman
Tokenizando archivo: 4 - AS - Teletrabajo-4k9
Tokenizando archivo: 4 - ICS - SoftwareDesign_PrincipiosyPatrones-Autentia
Tokenizando archivo: 4 - TA - Guía 1. Sistemas de control automático
Tokenizando archivo: 1 - SyO - 04 gestion-por-procesos
Tokenizando archivo: 1 - AyED - Unidad3 (7929)
Tokenizando archivo: 4 - AS - Gestión CPD
Tokenizando archivo: 5 - SSI - M5-Privacidad
Tokenizando archivo: 3 - BD - caselli_manual-de-base-de-datos-2019
Tokenizando archivo: 4 - UXUI - Diseño UX UNIDAD 1
Tokenizando archivo: 4 - AS - Teletrabajo-4k10
Tokenizando archivo: 4 - AS

### Tokenización con ventana deslizante
Una vez que se tienen los archivos con los tokens, deseamos **detectar conceptos adicionales**. Aquellos conceptos que adquieren significado con la combinación de palabras.

Para ello se utilizará un **algoritmo de ventana deslizante** con un `window_size = 4`. Lo que buscamos es dada una secuencia de tokens `["sistemas", "operativos", "distribuídos"]`formar los siguientes conceptos: `"sistemas operativos"`, `"sistemas operativos distribuídos"`.

In [3]:
from itertools import combinations

window_size = 4

In [ ]:
related_table = {}

for f in raw_files:
    print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

    tokens = []

    with open(f"{tokens_path}/{f}.txt", "rb") as tf:
        tokens = tf.read().decode("utf-8").split("\n")

    for i in range(len(tokens) - window_size):
        window = tokens[i:i+window_size]

        def get_subarrays(arr):
            result = []
            n = len(arr)
            for r in range(1, n+1):  # sizes from 1 to n
                for indices in combinations(range(n), r):
                    subarray = [arr[i] for i in indices]

                    result.append(subarray)
            return result
        
        arrays = get_subarrays(window[1:])

        # arrays.insert(0, []) # Permite formar conceptos de una sola palabra

        for arr in arrays:
            arr.insert(0, window[0])
            arr = [s.lower() for s in arr]

            if any(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', s) for s in arr):
                continue

            if len(arr) != len(set(arr)):
                continue
            # TODO: refactor this

            arr.sort()
            
            t = tuple(arr)
            if (not t in related_table):
                related_table[t] = 0
            related_table[t] += 1

print("\033[92mCantidad de conceptos candidatos:" + str(len(related_table)) + "\033[0m")

Detectando conceptos en archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Detectando conceptos en archivo: 1 - AyED - cpp según yo pero en pedo
Detectando conceptos en archivo: 1 - AyED - cpp según yo
Detectando conceptos en archivo: 1 - AyED - Unidad3 (7929)
Detectando conceptos en archivo: 1 - AyED - Unidad4 (7930)
Detectando conceptos en archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Detectando conceptos en archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Detectando conceptos en archivo: 1 - SyO - 01 Evolucion de las estructuras
Detectando conceptos en archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Detectando conceptos en archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Detectando conceptos en archivo: 1 - SyO - 04 gestion-por-procesos
Detectando conceptos en archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Detectando conceptos en archivo: 1 - SyO - 1)

### Creación Vocabulario
Para crear el vocabulario, se realizará un filtro al diccionario `related_table` resultante de los pasos anteriores. Definimos los parámetros `min_freq` y `max_freq` para ello.

In [23]:
def token_iterator(tokens_path):
    for file in os.listdir(tokens_path):
        with open(f"{tokens_path}/{file}", "rb") as tf:
            tokens = tf.read().decode("utf-8").split("\n")
            yield tokens

In [25]:
from collections import Counter
from typing import Iterator

class Vocab:
    def __init__(self, tokens, min_freq=50, max_freq=100, reserved_tokens = []):

        counter = Counter()
        if isinstance(tokens, Iterator):
            for token_batch in tokens:
                counter.update(token_batch)
        else:
            counter.update(tokens)

        self.token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)

        self.idx_to_token = list(sorted(set(['<unk>'] + reserved_tokens + [
            token for token, freq in self.token_freqs if freq >= min_freq and freq <= max_freq])))

        self.token_to_idx = {token: idx
                            for idx, token in enumerate(self.idx_to_token)}
        
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indexes):
        if hasattr(indexes, '__len__') and len(indexes) > 1:
            return [self.idx_to_token[int(index)] for index in indexes]
        return self.idx_to_token(indexes)
    
    @property
    def unk(self):
        return self.token_to_idx['<unk>']

In [29]:
tokens_path = "./data/tokens"
vocabulary = Vocab(token_iterator(tokens_path), min_freq=50, max_freq=100)
print("\033[92mTamaño del vocabulario: " + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario: 2575


En este punto, ya hemos seleccionado conjuntos de tokens que suelen aparecer cerca.

Estos conjuntos serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo de /data/plain mediante una ventana deslizante del mismo tamaño utilizado para detectar conceptos, separando en palabras siempre y cuando no se encuentre dentro de la ventana las palabras de un concepto.

Se generarán nuevos tokens, siendo estos numéricos (/data/tokens_num). LLos positivos (o 0) corresponden al índice de un concepto en el vocabulario, mientras que los negativos indican la cantidad de tokens no reconocidos (\<unk\>). Esto se realizó de esta forma para ahorrar espacio y tiempo de procesamiento.

Si en una ventana se detectara más de un concepto, se agregarán todos los que se encuentre. Debido al procesamiento que se realizará más adelante, no debería importar el orden.

In [4]:
import ast

concepts_file = "./data/vocabulary.txt"
vocabulary = []

with open(concepts_file, "rb") as cf:
    lines = cf.read().decode("utf-8").split("\n")[:-1]
    vocabulary = [ast.literal_eval(l) for l in lines]

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:2726


In [9]:
import math

tokens_conceptos_path = "./data/tokens_conceptos"

metrics_2 = {}

window_size_concept_tokenization = window_size * 3

for f in raw_files:
    print("\033[Tokenizando archivo por conceptos: " + f + "\033[0m")
    metrics_2[f] = {}
    found_concepts = 0

    recent_concepts = {}

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = [t.text for t in esp.tokenizer(txt)]

        metrics_2[f]["tokens"] = len(tokens)

        with open(tokens_conceptos_path + "/" + f + ".txt", "wb") as tnf:

            unks = 0

            for i in range(len(tokens) - window_size_concept_tokenization):
                window = tokens[i:i+window_size_concept_tokenization]
                for k, v in recent_concepts.items():
                    if v > 0:
                        recent_concepts[k] -= 1

                unks += 1
            
                for ix, concept in enumerate(vocabulary):
                    if all(word in window for word in concept) and (not ix in recent_concepts or recent_concepts[ix] == 0):
                        tnf.write(("-" + str((unks-1)) + " " + str(ix) + " ").encode("utf-8"))
                        unks = 0
                        recent_concepts[ix] = window_size_concept_tokenization
                        found_concepts += 1

                    
    metrics_2[f]["concepts"] = found_concepts

okenizando archivo por conceptos: 3 - AP - AdProy_2_Trabajo en Equipo_2022
okenizando archivo por conceptos: 3 - AP - respuestas
okenizando archivo por conceptos: 3 - BD - caselli_manual-de-base-de-datos-2019
okenizando archivo por conceptos: 3 - BD - Guía 1
okenizando archivo por conceptos: 3 - BD - Guía 2
okenizando archivo por conceptos: 3 - CD - capitulo2
okenizando archivo por conceptos: 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings
okenizando archivo por conceptos: 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion
okenizando archivo por conceptos: 3 - DS - Actor. Definicion. Clasificacion (1)
okenizando archivo por conceptos: 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados
okenizando archivo por conceptos: 3 - DS - Libro UML y Patrones - Larman
okenizando archivo por conceptos: 3 - DS - MerFNConceptos
okenizando archivo por conceptos: 4 - AS - Analisis PEST
okenizando archivo por conceptos: 4 - AS - Backups_raids
oken

In [10]:
print("\033[92m")
for archivo, item in metrics_2.items():
    print("Archivo " + archivo + ":")
    for nombre, valor in item.items():
        print("\t" + nombre + ": " + str(valor))
print("\033[0m")


Archivo 3 - AP - AdProy_2_Trabajo en Equipo_2022:
	tokens: 27662
	concepts: 275
Archivo 3 - AP - respuestas:
	tokens: 0
	concepts: 0
Archivo 3 - BD - caselli_manual-de-base-de-datos-2019:
	tokens: 28732
	concepts: 388
Archivo 3 - BD - Guía 1:
	tokens: 11365
	concepts: 390
Archivo 3 - BD - Guía 2:
	tokens: 9548
	concepts: 143
Archivo 3 - CD - capitulo2:
	tokens: 10412
	concepts: 236
Archivo 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings:
	tokens: 455695
	concepts: 17102
Archivo 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion:
	tokens: 530998
	concepts: 19677
Archivo 3 - DS - Actor. Definicion. Clasificacion (1):
	tokens: 1490
	concepts: 33
Archivo 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados:
	tokens: 4415
	concepts: 41
Archivo 3 - DS - Libro UML y Patrones - Larman:
	tokens: 253057
	concepts: 3868
Archivo 3 - DS - MerFNConceptos:
	tokens: 2342
	concepts: 42
Archivo 4 - AS - Analisis PEST:
	tokens: 1004
	concepts: 4
Arc

A partir de los tokens numéricos, se iterará por cada secuencia de token con un nuevo tamaño de ventana, mayor, tratando de distinguir conceptos relacionados.

Esta ventana se centrará en cada token (no -1), almacenando en un diccionario el token central, los tokens en el contexto y ejemplos negativos (para evitar que la red neuronal, al entrenar, aprenda que todos los tokens siempre están relacionados).

In [ ]:
class ConceptDataset(Dataset):
    def __init__(self, tokens_path, vocab_size, window_size=5, negative_samples=5):
        self.tokens_path = tokens_path
        self.window_size = window_size
        self.negative_samples = negative_samples
        self.vocab_size = vocab_size

        self.sequences = []
        for file in os.listdir(self.tokens_path):
            if not file.endswith(".txt"):
                continue
            with open(f"{self.tokens_path}/{file}", "rb") as tf:
                sequence = []
                for token in f.read.strip().split():
                    if token.startswith("-"):
                        # skip unkown tokens
                        continue
                    sequence.append(int(token))
                self.sequences.append(sequence)

            self.flat_sequences = [item for sublist in self.sequences for item in sublist]
            self.length = len(self.flat_sequences)



    def __len__(self):
        return self.le
    
    def __getitem__(self, idx):
        with open(f"{self.plain_path}/{idx}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = spacy.tokenizer(txt)
            with open(f"{self.tokens_path}/{idx}.txt", "wb") as tf:
                for token in tokens:
                    if (is_clean_token(token)):
                        tf.write((token.text + "\n").encode("utf-8"))

    def is_clean_token(token):
        return not (
            token.is_punct or
            token.is_space or
            token.is_stop or
            len(token.text) == 1)

    for f in raw_files:
        print("\033[94mTokenizando archivo: " + f + "\033[0m")

        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = spacy.tokenizer(txt)
            with open(f"{tokens_path}/{f}.txt", "wb") as tf:
                for token in tokens:
                    if (is_clean_token(token)):
                        tf.write((token.text + "\n").encode("utf-8"))

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

A partir de este Dataset, a su vez, se generará un DataLoader.

In [ ]:
import torch

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionado.

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [ ]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento:

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función:

Algunos ejemplos de la misma serían: